### Only for format  the raw data.

In [3]:
import pandas as pd

#### Load and format dataset Titanic survived dataset. 

In [4]:
# laod original trainning dataset.
train_data = pd.read_csv(
    "train.csv",
    sep=r'\s*,\s*',
    quotechar="'",
    engine='python',
    index_col=False,
    na_values="?")

# Counting...
Target = 'Survived'
Labels = train_data[Target].unique()
counts = train_data[Target].value_counts()
print(counts)

# laod original testing dataset (dataset without labels).
test_data = pd.read_csv(
    "test.csv",
    sep=r'\s*,\s*',
    engine='python',
    quotechar="'",
    index_col=False,
    na_values="?")

# Load labels for testing dataset.
test_labels = pd.read_csv(
    "gender_submission.csv",
    sep=r'\s*,\s*',
    engine='python',
    na_values="?")

# Counting...
Target = 'Survived'
Labels = test_labels[Target].unique()
counts = test_labels[Target].value_counts()
print(counts)

0    549
1    342
Name: Survived, dtype: int64
0    266
1    152
Name: Survived, dtype: int64


In [5]:
# Fusion labels to fusion train and test.
test_data['Survived'] = test_labels['Survived']

In [6]:
# Fusion the datasets using only selected features.
final = pd.concat([train_data[["Pclass","Sex","Age","SibSp","Parch","Fare","Embarked","Survived"]], test_data[["Pclass","Sex","Age","SibSp","Parch","Fare","Embarked","Survived"]]])

In [7]:
# Shuffling the data
final = final.sample(frac=1)
# Drop missing data
final = final.dropna()
# Convert to integer
final['Age'] = final['Age'].astype('float').astype('int64')
final['Fare'] = final['Fare'].astype('float').astype('int64')
# Converto to categorical features
final = final.astype('category')

In [143]:
# How much samples remaing.
Target = 'Survived'
Labels = final[Target].unique()
counts = final[Target].value_counts()
print(counts)

0    628
1    415
Name: Survived, dtype: int64


In [144]:
# Split the data by labels.
final0 = final.loc[final['Survived'] == 0]
final1 = final.loc[final['Survived'] == 1]
final0.reset_index(inplace=True)
final1.reset_index(inplace=True)

In [149]:
# A function to use 'frac' parameter.
# Split the data in frac < 1 for trainning and (1 - frac) for testing.

def makedata(final0,final1,frac):
    side0A = final0.loc[1:int(round(final0['index'].count()*frac)),:]
    side0B = final0.loc[int(round(final0['index'].count()*frac)):,:]
    side1A = final1.loc[1:int(round(final1['index'].count()*frac)),:]
    side1B = final1.loc[int(round(final1['index'].count()*frac)):,:]
    train = [side0A, side1A]
    test = [side0B, side1B]
    tempA = pd.concat(train).sample(frac=1)
    tempA.reset_index(inplace=True)
    tempB = pd.concat(test).sample(frac=1)
    tempB.reset_index(inplace=True)
    tempA = tempA.drop(tempA.columns[0:2], axis=1)
    tempB = tempB.drop(tempB.columns[0:2], axis=1)
    tempA = tempA.astype('str')
    tempB = tempB.astype('str')
    return tempA, tempB

In [150]:
train, test = makedata(final0,final1,0.6)

In [151]:
# Counting...

Target = 'Survived'
Labels = train[Target].unique()
counts = train[Target].value_counts()
print(counts)

Target = 'Survived'
Labels = test[Target].unique()
counts = test[Target].value_counts()
print(counts)

0    377
1    249
Name: Survived, dtype: int64
0    251
1    166
Name: Survived, dtype: int64


In [153]:
# Save the dataset in CSV.
train.to_csv('dataset.training.csv.titanic', sep=',',  encoding='ascii', decimal='.', index=False, header=False)
test.to_csv('dataset.test.csv.titanic', sep=',',  encoding='ascii', decimal='.', index=False, header=False)